In [1]:
from pathlib import Path
import itertools
from matplotlib import pyplot as plt
import pandas as pd
import re

In [2]:
# Set path to folder with faces: stim_data_dir
stim_data_dir = Path("..", "data", "Nimstim faces")

In [3]:
def create_csv(path=stim_data_dir, write=False):
    """ Create dataframe with all image IDs and sex, emotion, and mouth position depicted on image"""

    # Read all images
    files_path = list(path.glob('*.bmp'))
    files = list(x.stem for x in files_path)

    # Create codebook with the meaning of all abbreviations
    codebook = {0: {'female', 'male'},
           1: {'calm', 'angry', 'happy', 'fear', 'sad', 'surprised', 'neutral', 'disgust'},
           2: {'open', 'closed', 'exuberant'}}
    output = [[], [], []]

    # Decode sex, emotion, and mouth position depicted on image
    for file in files:
        file_parts = file.split('_')
        file_parts[0] = file_parts[0][-1]

        for i in range(0, 3):
            try:
                code = re.findall('(?<=\')' + file_parts[i].lower() + '\w*', str(codebook[i]))[0]
                output[i].append(code)
            except:
                if file_parts[i].lower() == 'sp':
                    output[i].append('surprised')
                elif file_parts[i].lower() == 'x':
                    output[i].append('exuberant')
                else:
                    next

    # Create dataframe with all collected info about the image
    df = {'file': files_path,
          'sex': output[0],
          'emotion': output[1],
          'mouth': output[2]}
    df = pd.DataFrame(df)
    df.index.name = 'picture_id'
    
    # Write dataframe to csv file
    if write:
        df.to_csv('overview.csv', index=True)
    
    return df


In [4]:
# Create overview of all images with their corresponding sex, emotion and mouth positioning
overview = create_csv(write=True)
overview

,file,sex,emotion,mouth
picture_id,,,,
0,../data/Nimstim faces/03F_HA_X.bmp,female,happy,exuberant
1,../data/Nimstim faces/05F_CA_C.bmp,female,calm,closed
2,../data/Nimstim faces/01F_CA_C.bmp,female,calm,closed
3,../data/Nimstim faces/06F_CA_C.bmp,female,calm,closed
4,../data/Nimstim faces/26M_AN_C.bmp,male,angry,closed
...,...,...,...,...
110,../data/Nimstim faces/05F_SA_O.bmp,female,sad,open
111,../data/Nimstim faces/32M_AN_C.bmp,male,angry,closed
112,../data/Nimstim faces/01F_FE_C.bmp,female,fear,closed


In [5]:
from protosc.preprocessing import GreyScale, ViolaJones, CutCircle
from protosc.pipe_complex import PipeComplex
from protosc.feature_extraction import FourierFeatures
from protosc.io import ReadImage
grayscale = True

In [6]:
# Define pipeline
pipe1 = ReadImage()*ViolaJones(20)*CutCircle()*FourierFeatures()
pipe2 = ReadImage()*GreyScale()*ViolaJones(20)*CutCircle()*FourierFeatures()

pipe_complex = pipe1 + pipe2

In [7]:
def execute_pipeline(sex=False, emotion=False, mouth=False):
    """ Put specified files through the pipeline"""
    
    overview = create_csv()
    
    if sex:
        overview = overview[overview['sex'] == sex]
    if emotion:
        overview = overview[overview['emotion'] == emotion]
    if mouth:
        overview = overview[overview['mouth'] == mouth]
    
    files = list(overview['file'])
    
    images = pipe_complex.execute(files)
    
    return images

In [8]:
# Put selected images through pipeline
output = execute_pipeline(sex='female', emotion='angry', mouth='open')

In [9]:
# Create Fourier Matrix: row = image, column = Feature
matrix = pd.DataFrame()
for row in range(len(output)):
    snip = output[row]['ReadImage+GreyScale+ViolaJones_20+CutCircle+FourierFeatures_a8s7']
    for col in range(len(snip)):
        matrix.loc[row, col] = snip[col]
matrix

KeyError: 'ReadImage+GreyScale+ViolaJones_20+CutCircle+FourierFeatures_a8s7'